# Solutions to Problem Set 0 — Coding Part

**Course:** Data Compression With And Without Deep probabilistic models (Prof. Bamler at University of Tuebingen)

- This notebook shows suggested solutions to the coding part of Problem Set 0, published on 21 April 2022 and discussed on 22 April 2022.
- Download the original problem set from the [course website](https://robamler.github.io/teaching/compress22/).

## Problem 0.3: Naive Symbol Code Implementation

In this exercise, we'll implement a very naive but correct encoder and decoder for prefix-free symbol codes.
We only care about correctness for now, not about computational efficiency.

We represent bit strings (code words and the concatenated encoded message) as lists of boolean values, where `True` represents a "one"-bit and `False` represents a "zero" bit.
Please be aware that this would be an extremely inefficient representation for a real application.
We represent code books as dictionaries from symbols to bit strings (i.e., to lists of boolean values).

### Sample Code Books

Our decoding algorithm will only work with prefix codes.
Let's define some sample prefix codes for our unit tests.

In [1]:
# our example C^{(4)} from Problem 1.2
SAMPLE_CODEBOOK_MONOPOLY_C4 = {
    2: [False, True, False],
    3: [True, False],
    4: [False, False],
    5: [True, True],
    6: [False, True, True]
}

# additional example (exercise: verify that this is a prefix code)
SAMPLE_CODEBOOK2 = {
    'a': [True, False],
    'b': [False],
    'c': [True, True, False, False],
    'd': [True, True, False, True],
    'e': [True, True, True],
}

### Encoder

The encoder is very simple.
Fill in the blank where it says "TODO" (a single line of code will do).

In [2]:
def encode(message, codebook):
    """Encodes a sequence of symbols using a prefix-free symbol code.

    This is a very inefficient implementation for teaching purposes only.

    Args:
        message (list): The message you want to encode, as a list of symbols.
        codebook (dict): A codebook for a prefix-free symbol code. Must be a
            dictionary whose keys contain all symbols that appear in `message`
            (and may contain additional keys). Each key must map to a list of
            booleans, representing the code word as a sequence of bits. Must
            specify a prefix-free code, i.e., no code word may be the prefix
            of the code word for a different symbol.

    Returns:
        list: The encoded bit string as a list of bools.
    """
    
    encoded = []
    
    for symbol in message:
        # TODO: look up code word for `symbol` in the `codebook` and append
        # it to `encoded`
        #
        # PROPOSED SOLUTION:
        encoded += codebook[symbol]
    
    return encoded

Now run these unit tests to verify your implementation:

In [3]:
assert encode([], SAMPLE_CODEBOOK_MONOPOLY_C4) == []
assert encode([], SAMPLE_CODEBOOK2) == []
assert (
    encode([4, 3, 6, 4, 2], SAMPLE_CODEBOOK_MONOPOLY_C4)
    == [False, False, True, False, False, True, True, False, False, False, True, False]
)
assert (
    encode(['c', 'b', 'a', 'd', 'b', 'b', 'd', 'e'], SAMPLE_CODEBOOK2)
    == [True, True, False, False, False, True, False, True, True, False, 
        True, False, False, True, True, False, True, True, True, True]
)

### Decoder

The decoder is more complicated because it has to infer the boundaries between concatenated code words.
To do this, we will use the assumption that the code book defines a *prefix-free* symbol code.

We use a kind of brute-force implementation here.
It is correct but very inefficient.
We'll implement a more efficient method for a specific type of prefix codes on Problem Set 3.

Fill in the blanks where it says "TODO".

In [4]:
def decode(encoded, codebook):
    """Decodes a bitstring into a sequence of symbols using a prefix-free symbol code.

    This is a very inefficient implementation for teaching purposes only.

    Args:
        encoded (list): The compressed bit string as a list of bools.
        codebook (dict): A codebook for a prefix-free symbol code.

    Returns:
        list: The decoded message as a list of symbols.
    """
    
    # Strategy:
    # - Initialize two variables, `decoded` and `partial_codeword`, as empty
    #   lists. We'll use `partial_codeword` as an internal buffer and `decoded`
    #   to accumulate the decoded symbols, which we eventually return.
    # - We then iterate over the bits in `encoded` and copy one bit at a time
    #   from it to `partial_codeword` until `partial_codeword` matches one of
    #   the code words in the provided `codebook`. Once that's the case, we
    #   look up the corresponding symbol, append it to `decoded`, clear
    #   `partial_codeword`, and continue iterating over bits.
    decoded = []
    partial_codeword = []

    # For better performance, we create an `inverse_codebook` which maps from
    # code words back to symbols. You'll use it in the loop below. Note that the
    # codewords in the provided `codebook` are lists, which cannot be used as
    # keys in python dictionaries. That's why we turn these lists into tuples.
    inverse_codebook = {
        tuple(codeword): symbol
        for symbol, codeword in codebook.items()
    }
    
    for bit in encoded:
        # TODO: implement the loop body:
        # - append the current bit to `partial_codeword`:
        # - check if `partial_codeword` is in the `inverse_codebook` (you'll
        #   have to turn `partial_codeword` into a tuple again). If it is, then
        #   look up the corresponding symbol in the `inverse_codebook`, append
        #   it to `decoded`, and clear the `partial_codeword`.
        #
        # PROPOSED SOLUTION:
        partial_codeword.append(bit)
        try:
            symbol = inverse_codebook[tuple(partial_codeword)]
            decoded.append(symbol)
            partial_codeword = []
        except:
            pass # `partial_codeword` does not yet match any full codeword.

    assert partial_codeword == [], 'The provided bitstring is invalid.'
    return decoded

Now run these unit tests to verify your implementation:

In [5]:
assert decode([], SAMPLE_CODEBOOK_MONOPOLY_C4) == []
assert decode([], SAMPLE_CODEBOOK2) == []
assert decode(
    [False, False, True, False, False, True, True, False, False, False, True, False],
    SAMPLE_CODEBOOK_MONOPOLY_C4) == [4, 3, 6, 4, 2]
assert decode(
    [True, True, False, False, False, True, False, True, True, False, 
     True, False, False, True, True, False, True, True, True, True],
    SAMPLE_CODEBOOK2) == ['c', 'b', 'a', 'd', 'b', 'b', 'd', 'e']

### Round-Trip Tests

These entropy coding algorithms can contain very subtle errors that wouldn't show up in the minimal unit tests we've tested so far.
It is generally a good idea to implement more elaborate tests.
This is easy to do, now that you have both an encoder and a decoder: generate some long-ish sequence of random symbols.
Then encode and decode them and verify that the decoder reconstructs the original message.
Always remember the random number seed so that, if you find an error, you can start debugging.